<img src=images/HashiCorp_PrimaryLogo_Black_RGB.png width=150 align="right">
<img src=images/Acme.jpeg width=100 align="left">

<a id =1></a>
## Encryption as a Service / Cryptography as a Service / Transit Secrets Engine


## Business Values:
 * Enables security teams to foritify data during transit and at rest
 * Relieve the burden of proper encryption/decryption from application developers
 * Sign adn Verify data; generate hashes and HMACs of data and act as a source of random bytes
 * If an intrusion occurs, your data is encrypted with **AES 256-bit GCM** encryption
 * A centrally managed resource that can encrypt any data for the organization at scale
 * High performance - Vault will encrypt at higher rates (20K+ requests/sec) than the cloud providers solutions.
 
<img src=images/EncryptionAsaService.png>


### Vault Setup

### Prerequisites
* Running Vault Enterprise instance (in dev mode) 


In [2]:
vault -version

Vault v1.6.0 (7ce0bd9691998e0443bc77e98b1e2a4ab1e965d4)


In [1]:
export ZSH_THEM="avit"
export VAULT_ADDR=http://127.0.0.1:8200
export VAULT_TOKEN=root
export VAULT_SKIP_VERIFY=true

In [3]:
vault status

Key                Value
---                -----
Seal Type          shamir
Initialized        true
Sealed             true
Total Shares       1
Threshold          1
Unseal Progress    0/1
Unseal Nonce       n/a
Version            1.6.1+ent
Storage Type       raft
HA Enabled         true


In [9]:
vault login root

WARNING! The VAULT_TOKEN environment variable is set! This takes precedence
over the value set by this command. To use the value set by this command,
unset the VAULT_TOKEN environment variable or set it to the token displayed
below.

Success! You are now authenticated. The token information displayed below
is already stored in the token helper. You do NOT need to run "vault login"
again. Future Vault requests will automatically use this token.

Key                  Value
---                  -----
token                root
token_accessor       cSCbeb1g2ZPMa2X2YvvY0wkq
token_duration       ∞
token_renewable      false
token_policies       ["root"]
identity_policies    []
policies             ["root"]


---
### Enable the Transit Secret Engine


In [10]:
vault secrets enable transit

Success! Enabled the transit secrets engine at: transit/



### Create an Encryption Key Ring

In [11]:
vault write -f transit/keys/orders

Success! Data written to: transit/keys/orders


---
### Encrypt Secrets

##### Note: Plaintext must be **base64-encoded** before it can be encrypted.

In [12]:
PLAIN="This is my demo secret"
echo "Plaintext: "$PLAIN

Plaintext: This is my demo secret


In [13]:
PLAINTEXT64=$(base64 <<< "$PLAIN")
echo "Plaintext in base64: " $PLAINTEXT64

Plaintext in base64:  VGhpcyBpcyBteSBkZW1vIHNlY3JldAo=


In [19]:
vault write transit/encrypt/orders plaintext=$PLAINTEXT64

Key            Value
---            -----
ciphertext     vault:v1:IQ9FnXgCtMbiOo4xvIa1gE2RkcpvM9ROoJ1zaAJIj5GUZfYB1foqVZ+/thyKPjyrTYk2
key_version    1


Vault does **NOT** store any of this data.  The output you received is ciphertext.  You can store this ciphertext at the desired location (e.g. mysql database) or pass it to your application.

We'll copy the value of ciphertext and refer to it later as "\<ciphertext\>"
    
Here's another way to encrypt using API


In [21]:
CIPHERTEXT=$(curl -s --header "X-Vault-Token: ${VAULT_TOKEN}" \
       --request POST \
       --data '{"plaintext": "'"${PLAINTEXT64}"'"}' \
       http://127.0.0.1:8200/v1/transit/encrypt/orders | jq -r '.data.ciphertext')

echo "Ciphertext: ${CIPHERTEXT}"

Ciphertext: vault:v1:KgMQ3KKJxyC/vAdpCpbvk5i3r5UKiX5f4+4zllvFPSlo+nVsz+gUdACLHc/uSTaKt2a1


---
### Decrypt Ciphertext

In [29]:
vault write transit/decrypt/orders ciphertext="vault:v1:KgMQ3KKJxyC/vAdpCpbvk5i3r5UKiX5f4+4zllvFPSlo+nVsz+gUdACLHc/uSTaKt2a1"

Key          Value
---          -----
plaintext    VGhpcyBpcyBteSBkZW1vIHNlY3JldAo=


\
Take the above output `plaintext` and execute a command
```
    base64 --decode <<< "<plaintext>"
```

In [28]:
base64 --decode <<< "VGhpcyBpcyBteSBkZW1vIHNlY3JldAo="

This is my demo secret


In [44]:
DECIPHER=$(curl -s --header "X-Vault-Token: ${VAULT_TOKEN}" \
     --request POST \
     --data '{"ciphertext": "'"${CIPHERTEXT}"'"}' \
    http://127.0.0.1:8200/v1/transit/decrypt/orders | jq -r '.data.plaintext')


echo "Plaintext in Base 64 : ${DECIPHER}\n"
echo "Plaintext : "
echo "${DECIPHER}" | base64 --decode

Plaintext in Base 64 : VGhpcyBpcyBteSBkZW1vIHNlY3JldAo=

Plaintext : 
This is my demo secret




---
### Rotate the Encryption key

One of the benefits of using the Vault transit secrets engine is its ability to easily rotate encryption keys. Keys can be rotated manually by a human or by an automated process.

In [34]:
vault write -f transit/keys/orders/rotate

Success! Data written to: transit/keys/orders/rotate


In [36]:
vault write transit/encrypt/orders plaintext=$(base64 <<< "$PLAIN")

Key            Value
---            -----
ciphertext     vault:v2:lbSxRZm0ozj956s/brbnkAFxvzrFFTpfi7MtHDiu2SaNer5XAeITUfhBzYSpqElFzpj8
key_version    2


In [56]:
## Encrypt using the http api 
CIPHERTEXT2=$(curl -s --header "X-Vault-Token: ${VAULT_TOKEN}" \
       --request POST \
       --data '{"plaintext": "'"${PLAINTEXT64}"'"}' \
       http://127.0.0.1:8200/v1/transit/encrypt/orders | jq -r '.data.ciphertext')

echo "Ciphertext after Rotation: ${CIPHERTEXT2}\n"
echo "Cipherext before Rotation: ${CIPHERTEXT}"

Ciphertext after Rotation: vault:v2:EJ//JUMu7+Dk4Dbwbuv5kSEqQIpkoi/MKjdn0idxpHK+O87PV/zdv1f/UFXaGmSXZdrK

Cipherext before Rotation: vault:v1:KgMQ3KKJxyC/vAdpCpbvk5i3r5UKiX5f4+4zllvFPSlo+nVsz+gUdACLHc/uSTaKt2a1


##### Notice that the first ciphertext starts with " vault:v1: ". After rotating the encryption key, the ciphertext starts with " vault:v2: ". This indicates that the data gets encrypted using the latest version of the key after the rotation

&nbsp;
### Rewrap the older version of ciphertext


In [53]:
REWRAP_CIPHERTEXT=$(curl -s --header "X-Vault-Token: ${VAULT_TOKEN}" \
   --request POST \
   --data '{"ciphertext": "'"${CIPHERTEXT}"'" }' \
   http://127.0.0.1:8200/v1/transit/rewrap/orders | jq -r '.data.ciphertext')
   

echo "Original Ciphertext: ${CIPHERTEXT}\n"
echo "Rewrapped Ciphertext: ${REWRAP_CIPHERTEXT}\n"


Original Ciphertext: vault:v1:KgMQ3KKJxyC/vAdpCpbvk5i3r5UKiX5f4+4zllvFPSlo+nVsz+gUdACLHc/uSTaKt2a1

Rewrapped Ciphertext: vault:v2:qaTaBkB6pxCcU0ODTQIoqW4jRCMrErH9ZmuNXjvYC3RR16nZ2oTQ5JcobbCfJF5qQ0uK



#### Note that the rewrapped ciphertext has a **'v2'** in it. 

### Update Key Configuration

A Vault admin can update the encryption key configuration to specify the minimum version of ciphertext allowed to be decrypted,  
the minimum version of the key that can be used to encrypt the plaintext, and if the key is allowed to be deleted, etc. for _**increased security**_.

### Rotate keys a few times (more versions)

In [59]:
export VAULT_TOKEN=root
vault write -f transit/keys/orders/rotate
vault write -f transit/keys/orders/rotate
vault write -f transit/keys/orders/rotate
vault write -f transit/keys/orders/rotate
vault write -f transit/keys/orders/rotate

Success! Data written to: transit/keys/orders/rotate
Success! Data written to: transit/keys/orders/rotate
Success! Data written to: transit/keys/orders/rotate
Success! Data written to: transit/keys/orders/rotate
Success! Data written to: transit/keys/orders/rotate


### Read the order key information back

In [60]:
vault read transit/keys/orders

Key                       Value
---                       -----
allow_plaintext_backup    false
deletion_allowed          false
derived                   false
exportable                false
keys                      map[1:1608516881 2:1608517527 3:1608537373 4:1608537373 5:1608537373 6:1608537373 7:1608537373]
latest_version            7
min_available_version     0
min_decryption_version    1
min_encryption_version    0
name                      orders
supports_decryption       true
supports_derivation       true
supports_encryption       true
supports_signing          false
type                      aes256-gcm96


##### The current version of the key is 7.  There is no restriction about the minimum key version and any of the key versions can decrypt the data (min_decyption_version)

### Set a Minimum Decryption Version

In [61]:
vault write transit/keys/orders/config min_decryption_version=5

Success! Data written to: transit/keys/orders/config


### Read the new Key Configuration

In [62]:
vault read transit/keys/orders

Key                       Value
---                       -----
allow_plaintext_backup    false
deletion_allowed          false
derived                   false
exportable                false
keys                      map[5:1608537373 6:1608537373 7:1608537373]
latest_version            7
min_available_version     0
min_decryption_version    5
min_encryption_version    0
name                      orders
supports_decryption       true
supports_derivation       true
supports_encryption       true
supports_signing          false
type                      aes256-gcm96


In [64]:
vault write transit/keys/orders/config min_decryption_version=2

Success! Data written to: transit/keys/orders/config


&nbsp;
##### What happen to those ciphertext that were encrypted with v1, v2, or v3 of the key?
Answer:  Vault would refuse to decrypt the data as the key used is less than the minimum key version allowed.

---
### Encrypt data via web UI

Open a browser and visit http://127.0.0.1:8200


---
### Sign and Validate data
Consider a time where you want to ensure the data came from a trusted source.  You don't care who can read the data, but you care about the source of the data.   In such case, you use data signing instead of encryption.

Transit Secret Engine supports a number of key types and some support signing and signature verification.

| Key Type |  Description |
|----------|--------------|
| aes128-gcm96 | AES-GCM with a 128-bit AES key and a 96-bit nonce |
| aes256-gcm96 | AES-GCM with a 256-bit AES key and a 96-bit nonce (**default**) |
| chachca20-poly1305 | ChaCha20-Poly1305 with a 256-bit key |
| ed25519 | ED 25519 |
|ecdsa-p256 | ECDSA using the P-256 elliptic curve |
|ecdsa-p256 | ECDSA using the P-384 elliptic curve |
|ecdsa-p521 | ECDSA using the P-521 elliptic curve |
| rsa-2048 | 2048-bit RSA key |
| rsa-3072 | 3072-bit RSA key |
| rsa-4096 | 4096-bit RSA key 


see https://www.vaultproject.io/api-docs/secret/transit#create-key


In [66]:
vault write transit/keys/newsletter type="rsa-4096"

Success! Data written to: transit/keys/newsletter


In [69]:
vault write transit/sign/newsletter input=$(base64 <<< "$PLAIN")

Key            Value
---            -----
key_version    1
signature      vault:v1:Ztd2qOQiYHqq2uXud20UGNkkdq/MnCqkMlfBGtHrKvFs7C2iZvo9BKjb++2LX9ccyJnCiyh29B6fe5G3aXO2mSabT5+4L7FDWzjlnnczQMhQ0RleK7QwhbgM29RCVrUYE0dBiWH5nBdwWnvDUT0BftP7fec77HVkZIS3n4JsLwz9UivwSzucjRyBtfHgtpNBTP7rjOwwHHzy0mW1L0L2eLf3IlFr1+PKT/5OSeT81umTGF6TAF7vYE3XjOcizTN3AScw1QstUUvtlAp0WP3JrFKKaTGc9bLRFvbD7s0x6HRvnCNgPDA5tsGpbtHDrrcMcYbqHT5EAdCxD0fi/ACqcogBAI/SqxDR86Anc5othwptjZVJD2OYf2MDCuaL75n9cTeA8fTbZr7SHOd9AwdNQoCS0Z9WjDLNijgzYDZbYkwsUVxofyspV3scOuYOvGy7/hv43GHw4NyXNDRHKjAx3LNzwNofJ7U+M+1Lo7HANCjK85FqNrTTWzHb2JPwVGFu2/OtwTdVbWYfFUWiDLYm75R7goDWiM9V44v/OgKHd3JdY/HfMShP4r08ek+bk+IzeRhUuszNddyLHMlBg4MKrkHkx0pkJ2aOTyo/0FTWuEWMAEKDY//uAPTMcIIvGffgudgMWgg6TbwpxLAbxAHi7bHqo7/DilyFYllUJiLfnOI=


In [70]:
# replace <signature> with the real data
vault write transit/verify/newsletter input=$(base64 <<< "$PLAIN") \
   signature="vault:v1:Ztd2qOQiYHqq2uXud20UGNkkdq/MnCqkMlfBGtHrKvFs7C2iZvo9BKjb++2LX9ccyJnCiyh29B6fe5G3aXO2mSabT5+4L7FDWzjlnnczQMhQ0RleK7QwhbgM29RCVrUYE0dBiWH5nBdwWnvDUT0BftP7fec77HVkZIS3n4JsLwz9UivwSzucjRyBtfHgtpNBTP7rjOwwHHzy0mW1L0L2eLf3IlFr1+PKT/5OSeT81umTGF6TAF7vYE3XjOcizTN3AScw1QstUUvtlAp0WP3JrFKKaTGc9bLRFvbD7s0x6HRvnCNgPDA5tsGpbtHDrrcMcYbqHT5EAdCxD0fi/ACqcogBAI/SqxDR86Anc5othwptjZVJD2OYf2MDCuaL75n9cTeA8fTbZr7SHOd9AwdNQoCS0Z9WjDLNijgzYDZbYkwsUVxofyspV3scOuYOvGy7/hv43GHw4NyXNDRHKjAx3LNzwNofJ7U+M+1Lo7HANCjK85FqNrTTWzHb2JPwVGFu2/OtwTdVbWYfFUWiDLYm75R7goDWiM9V44v/OgKHd3JdY/HfMShP4r08ek+bk+IzeRhUuszNddyLHMlBg4MKrkHkx0pkJ2aOTyo/0FTWuEWMAEKDY//uAPTMcIIvGffgudgMWgg6TbwpxLAbxAHi7bHqo7/DilyFYllUJiLfnOI="

Key      Value
---      -----
valid    true


&nbsp;

---
#### Thank you.
<img src=images/HashiCorp_PrimaryLogo_Black_RGB.png width=100 align="left">